In [ ]:
#packages needed for the code
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pytensor.tensor as pt
import pandas as pd
import h5py

#Design of figures
plt.rcParams['figure.constrained_layout.use'] = True
az.style.use("arviz-darkgrid")

eigh = pt.vectorize(pt.linalg.eigh, '(m,m)->(m),(m,m)') #defines the calculated eigenvalues as a vectors


In [ ]:
def sorting(energys,zerolevel=0): #a function that sorts the energylevels
    #energys - the dataset of the different energylevels
    #zerolevel - the energylevel which is set to zero
    
    energys = np.sort(energys,axis=-1) #sorts the energylevels from the lowest to the highest
    
    #sets the chosen energylevel to zero, by defult the lowest energylevel, and normilize all energylevels to the zerolevel
    for a in range(len(energys)):
        energys[a,:] = energys[a,:] - energys[a,zerolevel]
    
    return energys

Data = h5py.File('Simulated_Data.hdf5', "r") #load the data
Energys = pd.DataFrame(Data["eigenv"]) #extract the energys
Energys = Energys.to_numpy()/(2*np.pi) #convert to numpy-array and from angular velocity to frequency

Energys_10 = sorting(Energys)          #sorts all 10 energylevels of the data (10-level system)
Energys_9  = sorting(Energys,1)[:,1:]  #sorts all energylevels, but removes the 0,0,0 state (9-level system)
Energys_3  = sorting(Energys,1)[:,1:4] #sorts the one-excitaiton manifold (3-level system)
Energys_6  = sorting(Energys,4)[:,4:]  #sorts the two-excitaiton manifold (6-level system)

omegac = np.linspace(2.5,7.5,1001) #sweep-parameter for the data [GHz]

#Plot of the sorted energylevels for the data
plt.title('Energylevels for the given data', size=18)
plt.plot(omegac, Energys_10)
plt.ylabel('Energy  [GHz]', size=18)
plt.xlabel(r'$ω_C$  [GHz]', size=18)
plt.xticks(size=16)
plt.yticks(size=16)
plt.show()

In [ ]:
#a function that calculates the eigenvalues of the 10x10-matrix representaion of the Hamiltonian model
def energys_10(omega1,omega2,omegac,g12,g1c,g2c,alpha1,alpha2,alphac):
    n = omegac.shape[0] #number of datapoints for which the eigenvalues are calculated
    H = pt.zeros((n,10,10)) #size of the matrix

    #The matrix-elements for the 10x10 matrix
    H = pt.set_subtensor(H[:,1,1],omega1)
    H = pt.set_subtensor(H[:,2,2],omega2)
    H = pt.set_subtensor(H[:,1,2],g12)
    H = pt.set_subtensor(H[:,2,1],g12)
    H = pt.set_subtensor(H[:,1,3],g1c)
    H = pt.set_subtensor(H[:,3,1],g1c)
    H = pt.set_subtensor(H[:,2,3],g2c)
    H = pt.set_subtensor(H[:,3,2],g2c)
    H = pt.set_subtensor(H[:,3,3],omegac)
    
    H = pt.set_subtensor(H[:,0,4],g12)
    H = pt.set_subtensor(H[:,0,5],g1c)
    H = pt.set_subtensor(H[:,0,6],g2c)
    H = pt.set_subtensor(H[:,4,0],g12)
    H = pt.set_subtensor(H[:,5,0],g1c)
    H = pt.set_subtensor(H[:,6,0],g2c)

    H = pt.set_subtensor(H[:,4,4],omega1+omega2)
    H = pt.set_subtensor(H[:,5,5],omega1+omegac)
    H = pt.set_subtensor(H[:,4,5],g2c)
    H = pt.set_subtensor(H[:,5,4],g2c)
    H = pt.set_subtensor(H[:,4,6],g1c)
    H = pt.set_subtensor(H[:,6,4],g1c)
    H = pt.set_subtensor(H[:,5,6],g12)
    H = pt.set_subtensor(H[:,6,5],g12)
    H = pt.set_subtensor(H[:,6,6],omega2+omegac)

    H = pt.set_subtensor(H[:,7,7],2*omega1+alpha1)
    H = pt.set_subtensor(H[:,8,8],2*omega2+alpha2)
    H = pt.set_subtensor(H[:,9,9],2*omegac+alphac)

    H = pt.set_subtensor(H[:,4,7],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,5,7],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,6,8],np.sqrt(2)*g2c)
    H = pt.set_subtensor(H[:,4,8],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,5,9],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,6,9],np.sqrt(2)*g2c)

    H = pt.set_subtensor(H[:,7,4],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,7,5],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,8,6],np.sqrt(2)*g2c)
    H = pt.set_subtensor(H[:,8,4],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,9,5],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,9,6],np.sqrt(2)*g2c)
    
    Eig = eigh(H)[0] #calculation of the eigenvalues

    Eig = pt.sort(Eig, axis=-1) #sort from lowest to highest
    
    #set the lowest eigenvalue to zero and normilize the other values to the zero-level
    Eig = pt.set_subtensor(Eig[:,1],Eig[:,1]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,2],Eig[:,2]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,3],Eig[:,3]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,4],Eig[:,4]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,5],Eig[:,5]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,6],Eig[:,6]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,7],Eig[:,7]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,8],Eig[:,8]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,9],Eig[:,9]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,0],Eig[:,0]-Eig[:,0])
    return Eig #reurns the sorted and normilized eigenvalues which represents the energylevels

In [ ]:
#a function that calculates the eigenvalues of the 9x9 matrix representaion of the Hamiltonian model (the 0,0,0 state is removed)
def energys_9(omega1,omega2,omegac,g12,g1c,g2c,alpha1,alpha2,alphac):
    n = omegac.shape[0] #number of datapoints for which the eigenvalues are calculated
    H = pt.zeros((n,9,9)) #size of the matrix

    #The matrix-elements for the 9x9 matrix
    H = pt.set_subtensor(H[:,0,0],omega1)
    H = pt.set_subtensor(H[:,1,1],omega2)
    H = pt.set_subtensor(H[:,0,1],g12)
    H = pt.set_subtensor(H[:,1,0],g12)
    H = pt.set_subtensor(H[:,0,2],g1c)
    H = pt.set_subtensor(H[:,2,0],g1c)
    H = pt.set_subtensor(H[:,1,2],g2c)
    H = pt.set_subtensor(H[:,2,1],g2c)
    H = pt.set_subtensor(H[:,2,2],omegac)

    H = pt.set_subtensor(H[:,3,3],omega1+omega2)
    H = pt.set_subtensor(H[:,4,4],omega1+omegac)
    H = pt.set_subtensor(H[:,3,4],g2c)
    H = pt.set_subtensor(H[:,4,3],g2c)
    H = pt.set_subtensor(H[:,3,5],g1c)
    H = pt.set_subtensor(H[:,5,3],g1c)
    H = pt.set_subtensor(H[:,4,5],g12)
    H = pt.set_subtensor(H[:,5,4],g12)
    H = pt.set_subtensor(H[:,5,5],omega2+omegac)

    H = pt.set_subtensor(H[:,3,4],np.sqrt(2)*g2c)
    H = pt.set_subtensor(H[:,4,3],np.sqrt(2)*g2c)
    H = pt.set_subtensor(H[:,3,5],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,5,3],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,4,5],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,5,4],np.sqrt(2)*g1c)

    H = pt.set_subtensor(H[:,6,6],2*omega1+alpha1)
    H = pt.set_subtensor(H[:,7,7],2*omega2+alpha2)
    H = pt.set_subtensor(H[:,8,8],2*omegac+alphac)

    H = pt.set_subtensor(H[:,3,6],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,4,6],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,5,7],np.sqrt(2)*g2c)
    H = pt.set_subtensor(H[:,3,7],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,4,8],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,5,8],np.sqrt(2)*g2c)

    H = pt.set_subtensor(H[:,6,3],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,6,4],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,7,5],np.sqrt(2)*g2c)
    H = pt.set_subtensor(H[:,7,3],np.sqrt(2)*g12)
    H = pt.set_subtensor(H[:,8,4],np.sqrt(2)*g1c)
    H = pt.set_subtensor(H[:,8,5],np.sqrt(2)*g2c)
    
    Eig = eigh(H)[0] #calculation of the eigenvalues

    Eig = pt.sort(Eig, axis=-1) #sort from lowest to highest
    
    #set the lowest eigenvalue to zero and normilize the other values to the zero-level
    Eig = pt.set_subtensor(Eig[:,1],Eig[:,1]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,2],Eig[:,2]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,3],Eig[:,3]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,4],Eig[:,4]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,5],Eig[:,5]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,6],Eig[:,6]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,7],Eig[:,7]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,8],Eig[:,8]-Eig[:,0])
    Eig = pt.set_subtensor(Eig[:,0],Eig[:,0]-Eig[:,0])
    return Eig #reurns the sorted and normilized eigenvalues which represents the energylevels

In [ ]:
#a function that calculates the eigenvalues of the 3x3 matrix representaion of the Hamiltonian model (one-excitation manifold)
def energys_3(omega1,omega2,omegac,g12,g1c,g2c):    
    n = omegac.shape[0] #number of datapoints for which the eigenvalues are calculated
    H_3 = pt.zeros((n,3,3)) #size of the matrix

    #The matrix-elements for the 3x3 matrix
    H_3 = pt.set_subtensor(H_3[:,0,0],omega1)
    H_3 = pt.set_subtensor(H_3[:,1,1],omega2)
    H_3 = pt.set_subtensor(H_3[:,0,1],g12)
    H_3 = pt.set_subtensor(H_3[:,1,0],g12)
    H_3 = pt.set_subtensor(H_3[:,0,2],g1c)
    H_3 = pt.set_subtensor(H_3[:,2,0],g1c)
    H_3 = pt.set_subtensor(H_3[:,1,2],g2c)
    H_3 = pt.set_subtensor(H_3[:,2,1],g2c)
    H_3 = pt.set_subtensor(H_3[:,2,2],omegac)

    Eig_3 = eigh(H_3)[0] #calculation of the eigenvalues
    
    Eig_3 = pt.sort(Eig_3, axis=-1) #sort from lowest to highest

    #set the lowest eigenvalue to zero and normilize the other values to the zero-level
    Eig_3 = pt.set_subtensor(Eig_3[:,1],Eig_3[:,1]-Eig_3[:,0])
    Eig_3 = pt.set_subtensor(Eig_3[:,2],Eig_3[:,2]-Eig_3[:,0])
    Eig_3 = pt.set_subtensor(Eig_3[:,0],Eig_3[:,0]-Eig_3[:,0])
    return Eig_3 #reurns the sorted and normilized eigenvalues which represents the energylevels
    
#a function that calculates the eigenvalues of the 6x6 matrix representaion of the Hamiltonian model (two-excitation manifold)
def energys_6(omega1,omega2,omegac,g12,g1c,g2c,alpha1,alpha2,alphac):
    n = omegac.shape[0] #number of datapoints for which the eigenvalues are calculated
    H_6 = pt.zeros((n,6,6)) #size of the matrix

    #The matrix-elements for the 6x6 matrix
    H_6 = pt.set_subtensor(H_6[:,0,0],omega1+omega2)
    H_6 = pt.set_subtensor(H_6[:,1,1],omega1+omegac)
    H_6 = pt.set_subtensor(H_6[:,0,1],g2c)
    H_6 = pt.set_subtensor(H_6[:,1,0],g2c)
    H_6 = pt.set_subtensor(H_6[:,0,2],g1c)
    H_6 = pt.set_subtensor(H_6[:,2,0],g1c)
    H_6 = pt.set_subtensor(H_6[:,1,2],g12)
    H_6 = pt.set_subtensor(H_6[:,2,1],g12)
    H_6 = pt.set_subtensor(H_6[:,2,2],omega2+omegac)

    H_6 = pt.set_subtensor(H_6[:,3,3],2*omega1+alpha1)
    H_6 = pt.set_subtensor(H_6[:,4,4],2*omega2+alpha2)
    H_6 = pt.set_subtensor(H_6[:,5,5],2*omegac+alphac)

    H_6 = pt.set_subtensor(H_6[:,0,3],np.sqrt(2)*g12)
    H_6 = pt.set_subtensor(H_6[:,1,3],np.sqrt(2)*g1c)
    H_6 = pt.set_subtensor(H_6[:,2,4],np.sqrt(2)*g2c)
    H_6 = pt.set_subtensor(H_6[:,0,4],np.sqrt(2)*g12)
    H_6 = pt.set_subtensor(H_6[:,1,5],np.sqrt(2)*g1c)
    H_6 = pt.set_subtensor(H_6[:,2,5],np.sqrt(2)*g2c)

    H_6 = pt.set_subtensor(H_6[:,3,0],np.sqrt(2)*g12)
    H_6 = pt.set_subtensor(H_6[:,3,1],np.sqrt(2)*g1c)
    H_6 = pt.set_subtensor(H_6[:,4,2],np.sqrt(2)*g2c)
    H_6 = pt.set_subtensor(H_6[:,4,0],np.sqrt(2)*g12)
    H_6 = pt.set_subtensor(H_6[:,5,1],np.sqrt(2)*g1c)
    H_6 = pt.set_subtensor(H_6[:,5,2],np.sqrt(2)*g2c)
    
    Eig_6 = eigh(H_6)[0] #calculation of the eigenvalues

    Eig_6 = pt.sort(Eig_6, axis=-1) #sort from lowest to highest

    #set the lowest eigenvalue to zero and normilize the other values to the zero-level
    Eig_6 = pt.set_subtensor(Eig_6[:,1],Eig_6[:,1]-Eig_6[:,0])
    Eig_6 = pt.set_subtensor(Eig_6[:,2],Eig_6[:,2]-Eig_6[:,0])
    Eig_6 = pt.set_subtensor(Eig_6[:,3],Eig_6[:,3]-Eig_6[:,0])
    Eig_6 = pt.set_subtensor(Eig_6[:,4],Eig_6[:,4]-Eig_6[:,0])
    Eig_6 = pt.set_subtensor(Eig_6[:,5],Eig_6[:,5]-Eig_6[:,0])
    Eig_6 = pt.set_subtensor(Eig_6[:,0],Eig_6[:,0]-Eig_6[:,0])
    return Eig_6 #reurns the sorted and normilized eigenvalues which represents the energylevels

In [ ]:
#a function that adds a normally distributed noice to the data
def noice(energys,noice_std=0.1):
    #energys - the different energyleveles
    #noice_std - the standard deviation off the noice [GHz]
    
    shape = np.shape(energys)
    return energys + np.random.normal(0, noice_std, shape) #returns the energylevels with the added noice

In [ ]:
omegac = np.linspace(2.5,7.5,1001)  #sweep-parameter for the data [GHz]

#Generated data of the energys from the different matrix-representations (all values are in GHz)
eigenvalues_10 = pm.draw(energys_10(4,4.5,omegac,0.005,0.05,0.05,-0.25,-0.25,-0.25)) #from the 10x10-matrix
eigenvalues_9  = pm.draw(energys_9(4,4.5,omegac,0.005,0.05,0.05,-0.25,-0.25,-0.25))  #from the 9x9-matrix
eigenvalues_3  = pm.draw(energys_3(4,4.5,omegac,0.005,0.05,0.05))                    #from the 3x3-matrix
eigenvalues_6  = pm.draw(energys_6(4,4.5,omegac,0.005,0.05,0.05,-0.25,-0.25,-0.25))  #from the 6x6-matrix

#Generated data with added noice
eigenvalues_10_noice = noice(eigenvalues_10,0.001)
eigenvalues_9_noice = noice(eigenvalues_9,0.001)
eigenvalues_3_noice = noice(eigenvalues_3,0.001)
eigenvalues_6_noice = noice(eigenvalues_6,0.001)

In [ ]:
with pm.Model() as model_10: #the Bayesian inferens model for the 10-level system
    #Normally distributed priors [GHz]
    g12 = pm.Normal(r'$g_{12}$',mu=0.005,sigma=0.005*0.1)
    g1c = pm.Normal(r'$g_{1C}$',mu=0.05,sigma=0.05*0.1)
    g2c = pm.Normal(r'$g_{2C}$',mu=0.05,sigma=0.05*0.1)
    
    omega1 = pm.Normal(r'$ω_1$',mu=4,sigma=4*0.01)
    omega2 = pm.Normal(r'$ω_2$',mu=4.5,sigma=4*0.01)
    
    alpha1 = pm.Normal(r'$\alpha_1$',mu=-0.25,sigma=0.025)
    alpha2 = pm.Normal(r'$\alpha_2$',mu=-0.25,sigma=0.025)
    alphac = pm.Normal(r'$\alpha_C$',mu=-0.25,sigma=0.025)

    #Uniformly distributed priors [GHz]
    #g12 = pm.Uniform(r'$g_{12}$',lower=0.001, upper=0.1)
    #g1c = pm.Uniform(r'$g_{1C}$',lower=0.01, upper=0.1)
    #g2c = pm.Uniform(r'$g_{2C}$',lower=0.01, upper=0.1)
    
    #omega1 = pm.Uniform(r'$ω_1$',lower=3.5, upper=4.5)
    #omega2 = pm.Uniform(r'$ω_2$',lower=4, upper=5)
    
    #alpha1 = pm.Uniform(r'$\alpha_1$',lower=-0.4, upper=-0.1)
    #alpha2 = pm.Uniform(r'$\alpha_2$',lower=-0.4, upper=-0.1)
    #alphac = pm.Uniform(r'$\alpha_C$',lower=-0.4, upper=-0.1)

    #Expected value
    mu = energys_10(omega1,omega2,omegac,g12,g1c,g2c,alpha1,alpha2,alphac) #calculated from the 10x10-matrix
    
    #Uncertainty of the data and model [GHz]
    sigma = 0.1
    
    #Likelihood (normally distributed) [GHz]
    likelihood = pm.Normal('likelihood',mu=mu,sigma=sigma,observed=Energys_10) #the observed data is the imported data
    #likelihood = pm.Normal('likelihood',mu=mu,sigma=sigma,observed=eigenvalues_10_Noice) #the observed data is the generated data

In [ ]:
with model_10: #sampling of the posteriors for the 10-level system using the MCMC-algorithm Slice
    #draws  - number of datapoints sampled
    #tune   - number of calcualted datapoints before the draws are sampled
    #chains - number of iterations the posteriors are sampled
    idata_10 = pm.sample(draws=5000,tune=10000,chains=4,cores=4, step=pm.step_methods.Slice()) #the sampled prior

#a plot of the sampled posteriors and their trace for the draws
az.plot_trace(idata_10, figsize=(15,18), var_names=[r'$g_{12}$', r'$g_{1C}$', r'$g_{2C}$', r'$ω_1$', r'$ω_2$', r'$\alpha_1$', r'$\alpha_2$', r'$\alpha_C$'])   
plt.show()
az.summary(idata_10, round_to=5) #a summary of the the mean value, standard deviation and more for the posteriors

In [ ]:
az.InferenceData.to_netcdf(idata_10, 'Posteriors_10x10') #saves the data of the sampled posterioris for the 10-level system

In [ ]:
with pm.Model() as model_9: #the Bayesian inferens model for the 9-level system
    #Normally distributed priors [GHz]
    g12 = pm.Normal(r'$g_{12}$',mu=0.005,sigma=0.005*0.1)
    g1c = pm.Normal(r'$g_{1C}$',mu=0.05,sigma=0.05*0.1)
    g2c = pm.Normal(r'$g_{2C}$',mu=0.05,sigma=0.05*0.1)
    
    omega1 = pm.Normal(r'$ω_1$',mu=4,sigma=4*0.01)
    omega2 = pm.Normal(r'$ω_2$',mu=4.5,sigma=4*0.01)
    
    alpha1 = pm.Normal(r'$\alpha_1$',mu=-0.25,sigma=0.025)
    alpha2 = pm.Normal(r'$\alpha_2$',mu=-0.25,sigma=0.025)
    alphac = pm.Normal(r'$\alpha_C$',mu=-0.25,sigma=0.025)

    #Uniformly distributed priors [GHz]
    #g12 = pm.Uniform(r'$g_{12}$',lower=0.001, upper=0.1)
    #g1c = pm.Uniform(r'$g_{1C}$',lower=0.01, upper=0.1)
    #g2c = pm.Uniform(r'$g_{2C}$',lower=0.01, upper=0.1)
    
    #omega1 = pm.Uniform(r'$ω_1$',lower=3.5, upper=4.5)
    #omega2 = pm.Uniform(r'$ω_2$',lower=4, upper=5)
    
    #alpha1 = pm.Uniform(r'$\alpha_1$',lower=-0.4, upper=-0.1)
    #alpha2 = pm.Uniform(r'$\alpha_2$',lower=-0.4, upper=-0.1)
    #alphac = pm.Uniform(r'$\alpha_C$',lower=-0.4, upper=-0.1)

    #Expected value
    mu = energys_9(omega1,omega2,omegac,g12,g1c,g2c,alpha1,alpha2,alphac) #calculated from the 9x9-matrix
    
    #Uncertainty of the data and model [GHz]
    sigma = 0.1
    
    #Likelihood (normally distributed) [GHz]
    likelihood = pm.Normal('likelihood',mu=mu,sigma=sigma,observed=Energys_9) #the observed data is the imported data
    #likelihood = pm.Normal('likelihood',mu=mu,sigma=sigma,observed=eigenvalues_9_Noice) #the observed data is the generated data

In [ ]:
with model_9: #sampling of the posteriors for the 9-level system using the MCMC-algorithm Slice
    #draws  - number of datapoints sampled
    #tune   - number of calcualted datapoints before the draws are sampled
    #chains - number of iterations the posteriors are sampled
    idata_9 = pm.sample(draws=5000,tune=10000,chains=4,cores=4, step=pm.step_methods.Slice()) #the sampled prior

#a plot of the sampled posteriors and their trace for the draws
az.plot_trace(idata_9, figsize=(15,18), var_names=[r'$g_{12}$', r'$g_{1C}$', r'$g_{2C}$', r'$ω_1$', r'$ω_2$', r'$\alpha_1$', r'$\alpha_2$', r'$\alpha_C$'])   
plt.show()
az.summary(idata_9, round_to=5) #a summary of the the mean value, standard deviation and more for the posteriors

In [ ]:
az.InferenceData.to_netcdf(idata_9, 'Posteriors_9x9') #saves the data of the sampled posterioris for the 9-level system

In [ ]:
with pm.Model() as model_3: #the Bayesian inferens model for the 3-level system
    #Normally distributed priors [GHz]
    g12 = pm.Normal(r'$g_{12}$',mu=0.005,sigma=0.005*0.1)
    g1c = pm.Normal(r'$g_{1C}$',mu=0.05,sigma=0.05*0.1)
    g2c = pm.Normal(r'$g_{2C}$',mu=0.05,sigma=0.05*0.1)
    
    omega1 = pm.Normal(r'$ω_1$',mu=4,sigma=4*0.01)
    omega2 = pm.Normal(r'$ω_2$',mu=4.5,sigma=4.5*0.01)

    #Uniform distributed priors [GHz]
    #g12 = pm.Uniform(r'$g_{12}$',lower=0.001, upper=0.01)
    #g1c = pm.Uniform(r'$g_{1C}$',lower=0.01, upper=0.1)
    #g2c = pm.Uniform(r'$g_{2C}$',lower=0.01, upper=0.1)
    
    #omega1 = pm.Uniform(r'$ω_1$',lower=3.5, upper=4.5)
    #omega2 = pm.Uniform(r'$ω_2$',lower=4, upper=5)

    #Expected value
    mu = energys_3(omega1,omega2,omegac,g12,g1c,g2c) #calculated from the 3x3-matrix
    
    #Uncertainty of the data and model [GHz]
    sigma = 0.1
    
    #Likelihood (normally distributed) [GHz]
    likelihood = pm.Normal('likelihood',mu=mu,sigma=sigma,observed=Energys_3) #the observed data is the imported data
    #likelihood = pm.Normal('likelihood',mu=mu,sigma=sigma,observed=eigenvalues_3_Noice) #the observed data is the generated data

In [ ]:
with model_3: #sampling of the posteriors for the 3-level system using the MCMC-algorithm Slice
    #draws  - number of datapoints sampled
    #tune   - number of calcualted datapoints before the draws are sampled
    #chains - number of iterations the posteriors are sampled
    idata_3 = pm.sample(draws=5000,tune=10000,chains=4,cores=4, step=pm.step_methods.Slice()) #the sampled prior

#a plot of the sampled posteriors and their trace for the draws
az.plot_trace(idata_3, figsize=(15,18), var_names=[r'$g_{12}$', r'$g_{1C}$', r'$g_{2C}$', r'$ω_1$', r'$ω_2$'])   
plt.show()
az.summary(idata_3, round_to=5) #a summary of the the mean value, standard deviation and more for the posteriors

In [ ]:
az.InferenceData.to_netcdf(idata_3, 'Posteriors_3x3') #saves the data of the sampled posterioris for the 3-level system

In [ ]:
with pm.Model() as model_6: #the Bayesian inferens model for the 6-level system
    #Normally distributed priors [GHz]
    g12 = pm.Normal(r'$g_{12}$',mu=0.005,sigma=0.005*0.1)
    g1c = pm.Normal(r'$g_{1C}$',mu=0.05,sigma=0.05*0.1)
    g2c = pm.Normal(r'$g_{2C}$',mu=0.05,sigma=0.05*0.1)
    
    omega1 = pm.Normal(r'$ω_1$',mu=4,sigma=4*0.01)
    omega2 = pm.Normal(r'$ω_2$',mu=4.5,sigma=4*0.01)
    
    alpha1 = pm.Normal(r'$\alpha_1$',mu=-0.25,sigma=0.025)
    alpha2 = pm.Normal(r'$\alpha_2$',mu=-0.25,sigma=0.025)
    alphac = pm.Normal(r'$\alpha_C$',mu=-0.25,sigma=0.025)

    #Uniformly distributed priors [GHz]
    #g12 = pm.Uniform(r'$g_{12}$',lower=0.001, upper=0.1)
    #g1c = pm.Uniform(r'$g_{1C}$',lower=0.01, upper=0.1)
    #g2c = pm.Uniform(r'$g_{2C}$',lower=0.01, upper=0.1)
    
    #omega1 = pm.Uniform(r'$ω_1$',lower=3.5, upper=4.5)
    #omega2 = pm.Uniform(r'$ω_2$',lower=4, upper=5)
    
    #alpha1 = pm.Uniform(r'$\alpha_1$',lower=-0.4, upper=-0.1)
    #alpha2 = pm.Uniform(r'$\alpha_2$',lower=-0.4, upper=-0.1)
    #alphac = pm.Uniform(r'$\alpha_C$',lower=-0.4, upper=-0.1)

    #Expected value
    mu = energys_6(omega1,omega2,omegac,g12,g1c,g2c,alpha1,alpha2,alphac) #calculated from the 6x6-matrix
    
    #Uncertainty of the data and model [GHz]
    sigma = 0.1
    
    #Likelihood (normally distributed) [GHz]
    likelihood = pm.Normal('likelihood',mu=mu,sigma=sigma,observed=Energys_6) #the observed data is the imported data
    #likelihood = pm.Normal('likelihood',mu=mu,sigma=sigma,observed=eigenvalues_6_Noice) #the observed data is the generated data

In [ ]:
with model_6: #sampling of the posteriors for the 6-level system using the MCMC-algorithm Slice
    #draws  - number of datapoints sampled
    #tune   - number of calcualted datapoints before the draws are sampled
    #chains - number of iterations the posteriors are sampled
    idata_6 = pm.sample(draws=5000,tune=10000,chains=4,cores=4, step=pm.step_methods.Slice()) #the sampled prior

#a plot of the sampled posteriors and their trace for the draws
az.plot_trace(idata_6, figsize=(15,18), var_names=[r'$g_{12}$', r'$g_{1C}$', r'$g_{2C}$', r'$ω_1$', r'$ω_2$', r'$\alpha_1$', r'$\alpha_2$', r'$\alpha_C$'])   
plt.show()
az.summary(idata_6, round_to=5) #a summary of the the mean value, standard deviation and more for the posteriors

In [ ]:
az.InferenceData.to_netcdf(idata_3, 'Posteriors_6x6') #saves the data of the sampled posterioris for the 6-level system